# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [3]:
import pandas as pd
import numpy as np

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [4]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
    !tar -xzvf data.tgz

## How many rows are there in the GBvideos.csv after removing duplications?

In [5]:
df = pd.read_csv('data/GBvideos.csv')
df_no_dup = df.drop_duplicates()
print(f"Rows before remove duplicate: {len(df)}")
print(f"Rows after remove duplicate: {len(df_no_dup)}")

Rows before remove duplicate: 38916
Rows after remove duplicate: 38745


## How many VDO's that contain at least one record (row) of "dislikes" more than "likes"?  <font color=red>DO NOT group by the data and make sure that you count only unique title!</font>

In [6]:
df_dis_more_like = df.loc[df['dislikes'] > df['likes'],"title"].unique()
print(f"Total of VDO's that contain at least one record (row) of \"dislikes\" more than \"likes\" is {len(df_dis_more_like)}")

Total of VDO's that contain at least one record (row) of "dislikes" more than "likes" is 56


## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [7]:
print(f"Total of VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments is {len(df[(df['comment_count']>10000)  & (df['trending_date'].str.contains('18.22.01'))])}")

Total of VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments is 28


## Which date that has the minimum average number of comments per VDO?

In [8]:
df['category_id']

0        26
1        24
2        10
3        17
4        25
         ..
38911    10
38912    10
38913    10
38914    24
38915    10
Name: category_id, Length: 38916, dtype: int64

In [9]:
from datetime import datetime
df_group_publish = df.groupby(by=['trending_date'])
df_date_min_avg = df_group_publish['comment_count'].mean().idxmin()
print(f"The date that has the minimum average number of comments per VDO is {datetime.strptime(df_date_min_avg, '%y.%d.%m').date()}")

The date that has the minimum average number of comments per VDO is 2017-11-15


## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

In [11]:
import json
from pandas import json_normalize
with open("data/GB_category_id.json") as file:
    json_data = json.load(file)
df_category_id = json_normalize(json_data, record_path=['items'])
sports_id = int(df_category_id.loc[(df_category_id["snippet.title"] == "Sports") & (df_category_id["snippet.assignable"]),"id"].iloc[0])
comedy_id = int(df_category_id.loc[(df_category_id["snippet.title"] == "Comedy") & (df_category_id["snippet.assignable"]),"id"].iloc[0])
df_sports = df[df['category_id'] == sports_id]
df_comedy = df[df['category_id'] == comedy_id]
df_sports_sum = df_sports.groupby(by=['trending_date'],as_index=False)['views'].sum()
df_comedy_sum = df_comedy.groupby(by=['trending_date'],as_index=False)['views'].sum()
df_sports_comp_comedy = pd.merge(df_sports_sum, df_comedy_sum, on='trending_date', how='outer')
df_sports_comp_comedy
# df_sports_comp_comedy = df_sports_comp_comedy[((df_sports_comp_comedy['trending_date'].notna()) & ((df_sports_comp_comedy['views_y'].isna()) | (df_sports_comp_comedy['views_x'] > df_sports_comp_comedy['views_y'])))]
# print(f"Total days that there are more total daily views of VDO in \"Sports\" category than in \"Comedy\" category is {len(df_sports_comp_comedy)}")

,trending_date,views_x,views_y
0,17.01.12,9879808,10889970
1,17.02.12,10856461,13302391
2,17.03.12,10875249,14259944
3,17.04.12,11399173,16617431
4,17.05.12,11261410,19089597
...,...,...,...
200,18.30.04,15114095,17802688
201,18.30.05,5153521,16427316
202,18.31.01,8431519,14840766
203,18.31.03,22546308,21528161
